# First Model Attempt

For this first model attempt, I trained a convolutional neural network using the InceptionV3 architecture from scratch. I could not use any pretrained weights for this version, as all of my images were fullsize in grayscale, making their dimensions incompatible with the full color versions of the models with pretrained weights. Additionally, in trying to convert the shape to use pretrained weights, my kernel would consistently crash due to the size of the images. I addressed these issues in my second version of the model. In training from scratch, I also decided to use a portion of the dataset rather than the entire thing to create balanced classes. 

In the end, due to my not modifying the sizes of the input images, the training took an extremely long time. For the sake of time (because this assignment is due in three days), I partially trained the model and found that my AUC score increased from 0.47 to 0.50 after 9 epochs. I expect that with further training that this score would increase, but for the sake of time and wanting to iterate on my model I had to stop here with this version.

In [1]:
# imports
import numpy as np
import tensorflow as tf

2023-03-21 08:31:05.762330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load data
Xtrain = np.load("../data/train/Xtrain.npy")
Ytrain = np.load("../data/train/Ytrain.npy")
Xtest = np.load("../data/test/Xtest.npy")
Ytest = np.load("../data/test/Ytest.npy")

In [3]:
# balance classes
Ytrain_use = Ytrain[:1124]
Xtrain_use = Xtrain[:1124]
Ytest_use = Ytest[:282]
Xtest_use = Xtest[:282]

## convert grayscale image to rgb images to make compatible with TF

In [4]:
# define image reshaper function to create rgb values for each pixel in the image
def img_reshaper(orig_img):
    new_img = []
    for row in orig_img:
        nrow = []
        for item in row:
            nrow.append([item, item, item])
        new_img.append(nrow)
    return new_img

In [ ]:
# reshape Xtrain
Xtrain_temp = []
c = 1
for img in Xtrain_use:
    Xtrain_temp.append(img_reshaper(img))
    print(f"{c} out of 1124 complete.")
    c+=1
Xtrain_new = np.array(Xtrain_temp)

1 out of 1124 complete.
2 out of 1124 complete.
3 out of 1124 complete.
4 out of 1124 complete.
5 out of 1124 complete.
6 out of 1124 complete.
7 out of 1124 complete.
8 out of 1124 complete.
9 out of 1124 complete.
10 out of 1124 complete.
11 out of 1124 complete.
12 out of 1124 complete.
13 out of 1124 complete.
14 out of 1124 complete.
15 out of 1124 complete.
16 out of 1124 complete.
17 out of 1124 complete.
18 out of 1124 complete.
19 out of 1124 complete.
20 out of 1124 complete.
21 out of 1124 complete.
22 out of 1124 complete.
23 out of 1124 complete.
24 out of 1124 complete.
25 out of 1124 complete.
26 out of 1124 complete.
27 out of 1124 complete.
28 out of 1124 complete.
29 out of 1124 complete.
30 out of 1124 complete.
31 out of 1124 complete.
32 out of 1124 complete.
33 out of 1124 complete.
34 out of 1124 complete.
35 out of 1124 complete.
36 out of 1124 complete.
37 out of 1124 complete.
38 out of 1124 complete.
39 out of 1124 complete.
40 out of 1124 complete.
41 out of

In [ ]:
# check shape for correctness
Xtrain_new.shape

In [ ]:
# save modified Xtrain and Ytrain
np.save("../data/train/Xtrain_final.npy", Xtrain_new)
np.save("../data/train/Ytrain_final.npy", Ytrain_use)

In [ ]:
# reshape Xtest
Xtest_temp = []
c = 1
for img in Xtest_use:
    Xtest_temp.append(img_reshaper(img))
    print(f"{c} out of 1124 complete.")
    c+=1
Xtest_new = np.array(Xtrain_temp)

In [ ]:
# check shape for correctness
Xtest_new.shape

In [ ]:
# save modified Xtest and Ytest
np.save("../data/train/Xtest_final.npy", Xtest_new)
np.save("../data/train/Ytest_final.npy", Ytest_use)

## modeling

In [4]:
# setup CNN using Inception-v3 (using google JAMA paper for this choice)
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights=None,
    input_shape=(Xtrain_use.shape[1],Xtrain_use.shape[2],1)
)

base_model.load_weights('/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

#base_model = PTModel(input_shape =  t_x.shape[1:], include_top = False, weights = 'imagenet')

2023-03-21 08:31:16.317359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# freeze the base model
base_model.trainable = False

In [6]:
# create new model that we are going to iterate on
inputs = tf.keras.Input(shape=(1776, 2368))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

In [7]:
# train!
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.Accuracy()])
model.fit(x=Xtrain_use.reshape((Xtrain_use.shape[0],Xtrain_use.shape[1],Xtrain_use.shape[2],1)), y=Ytrain_use, epochs=20, validation_data=(Xtest_use.reshape((Xtest_use.shape[0],Xtrain_use.shape[1],Xtrain_use.shape[2],1)), Ytest_use))

Epoch 1/20
36/36 [==============================] - 4641s 129s/step - loss: 0.7012 - auc: 0.4907 - accuracy: 0.0000e+00 - val_loss: 0.6934 - val_auc: 0.5000 - val_accuracy: 0.0000e+00
Epoch 2/20
 9/36 [======>.......................] - ETA: 1:17:35 - loss: 0.6944 - auc: 0.4808 - accuracy: 0.0000e+00